In [1]:
import pandas as pd
import numpy as np
from numpy import nan
import datetime
import requests
from IPython.display import JSON
import json
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps
import sqlalchemy
import pytz

In [2]:
OWM_key = "1b38d77f1df3e06fc0408e34db8ebee7"
city = "Cape Town"
country = "ZA"
forecast_intervals = '3h'
owm = OWM(OWM_key)
mgr = owm.weather_manager()

In [3]:
forecast = mgr.forecast_at_place(city + ',' + country, forecast_intervals)

def forecast_extraction(forecast_object): 
    """
    Given a forecast object from Open weathor Pyhton API, 
    return a structured dataframe. 

    :param: forecast object
    :returns: a dataframe with specific weather information
    """
    forecast_df = pd.json_normalize({
        'time' : forecast_object.reference_time(timeformat='date'), 
        'temperature' : forecast_object.temperature(unit='celsius'), 
        'precipitation_probabily' : forecast_object.precipitation_probability, 
        'rain' : forecast_object.rain, 
        'humidity' : forecast_object.humidity, 
        'status' : forecast_object.status, 
        'snow' : [nan if forecast_object.snow is None else forecast_object.snow], 
        'wind' : forecast_object.wind('km_hour')
    })

    return(forecast_df)

weather_forecast = pd.concat([forecast_extraction(weather) for weather in forecast.forecast.weathers])

In [4]:
def extract_fields(df):
    """
    Extract specific fields from the weather_forecast DataFrame.
    
    :param df: Input DataFrame containing weather forecast data.
    :return: DataFrame with selected fields.
    """
    selected_fields = df[['time', 'precipitation_probabily', 'humidity', 'status',
                          'temperature.temp', 'temperature.temp_min', 'temperature.temp_max',
                          'wind.speed', 'rain.3h']]
    return selected_fields

weather_forecast = extract_fields(weather_forecast)

In [5]:
def convert_to_sast(df):
    """
    Convert the datetime column from UTC to SAST (South Africa Standard Time).
    
    :param df: DataFrame with a datetime column in UTC.
    :return: DataFrame with the datetime column converted to SAST.
    """
    # Define the SAST time zone
    sast_timezone = pytz.timezone('Africa/Johannesburg') 
    
    # Convert the datetime column to SAST
    df['time'] = df['time'].dt.tz_convert(sast_timezone)
    
    return df

# Convert the datetime column to SAST
weather_forecast = convert_to_sast(weather_forecast)

In [6]:
def rename_columns(df):
    """
    Rename columns in the DataFrame to desired names.
    
    :param df: Input DataFrame with selected fields.
    :return: DataFrame with renamed columns.
    """
    renamed_df = df.rename(columns={
        'time': 'date',
        'temperature.temp': 'temperature',
        'temperature.temp_min': 'min_temp',
        'temperature.temp_max': 'max_temp',
        'wind.speed': 'wind_speed',
        'rain.3h': 'rain_3h'
    })
    return renamed_df

weather_forecast = rename_columns(weather_forecast)

### Ingest the data to MySQL

In [7]:
# connection details for the local MySQL database
schema = "weather"
host = "127.0.0.1"
user = "root"
password = "Li7329160_"
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [8]:
# send the weather data to the MySQL database
weather_forecast.to_sql('weather_data', if_exists = 'append', con = con, index=False)

40